## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Carillon North Tower\r6899 Collins Ave 3104\rM...,2/14/2023,76,"$3,995,000.00",2670,"$1,496.25",Alex Algarin,Compass Florida LLC,Alex Algarin,Compass Florida LLC,Carillon North Tower 6899 Collins Ave 3104 Mia...,Carillon North Tower,6899 Collins Ave 3104 Miami Beach,"(25.85381, -80.11977000000002)",25.853810,-80.119770
2,One Park Grove Condo\r2811 S Bayshore Dr 4B\rM...,2/15/2023,113,"$3,400,000.00",2543,"$1,337.00",David Siddons,Douglas Elliman,Robin Parker,Douglas Elliman,One Park Grove Condo 2811 S Bayshore Dr 4B Miami,One Park Grove Condo,2811 S Bayshore Dr 4B Miami,"(25.7284731, -80.2381924)",25.728473,-80.238192
3,Turnberry Ocean Colony\r16047 Collins Ave 1101...,2/15/2023,36,"$3,200,000.00",2772,"$1,154.40",Brittani Brookins,Coldwell Banker Realty,Maria Kazak,Kazak Realty,Turnberry Ocean Colony 16047 Collins Ave 1101 ...,Turnberry Ocean Colony,16047 Collins Ave 1101 Sunny Isles Beach,"(25.9252763, -80.1216546)",25.925276,-80.121655
4,Ritz Carlton Sunny Isles\r15701 Collins Ave 38...,2/15/2023,1,"$2,960,000.00",1447,"$2,045.61",Kelly Salinas,RE/MAX Advance Realty II,Michael Martirena,Compass Florida LLC,Ritz Carlton Sunny Isles 15701 Collins Ave 380...,Ritz Carlton Sunny Isles,15701 Collins Ave 3803 Sunny Isles Beach,"(25.922045, -80.12199729999999)",25.922045,-80.121997
5,Jade Beach\r17001 Collins Ave 3908\rSunny Isle...,2/16/2023,94,"$2,950,000.00",3211,$918.72,Lana Bell,Douglas Elliman,Lana Bell,Douglas Elliman,Jade Beach 17001 Collins Ave 3908 Sunny Isles ...,Jade Beach,17001 Collins Ave 3908 Sunny Isles Beach,"(25.9335244, -80.12091989999999)",25.933524,-80.120920
6,Waterside Towers Condo\r2450 NE 135 St 907\rNo...,2/15/2023,55,"$2,749,000.00",895,"$3,071.51",Mario Chirino,Douglas Elliman,Elian Garcia Hernandez,Canvas Real Estate,Waterside Towers Condo 2450 NE 135 St 907 Nort...,Waterside Towers Condo,2450 NE 135 St 907 North Miami,"(25.9000175, -80.1495267)",25.900018,-80.149527
7,The Ocean Club\r785 Crandon Blvd 1105\rKey Bis...,2/17/2023,51,"$2,510,000.00",2100,"$1,195.24",Douglas Kinsley,Fortune Christie's Int'l Real Estate,Lucia Marin,One Sotheby's Int'l Realty,The Ocean Club 785 Crandon Blvd 1105 Key Biscayne,The Ocean Club,785 Crandon Blvd 1105 Key Biscayne,"(25.6875547, -80.15912399999999)",25.687555,-80.159124
8,Space 01\r7934 West Dr 501\rNorth Bay Village,2/14/2023,73,"$2,400,000.00",3721,$644.99,Vicky Rua,Compass Florida LLC,Ramon Dominguez,Lifestyle International Realty,Space 01 7934 West Dr 501 North Bay Village,Space,01 7934 West Dr 501 North Bay Village,"(25.8526596, -80.1603678)",25.852660,-80.160368
9,Ritz Carlton Grove Residence\r3400 SW 27 Ave 1...,2/15/2023,31,"$2,300,000.00",2110,"$1,090.05",Marilyn Cromer,Douglas Elliman,Glen Janney,Cervera Real Estate Inc.,Ritz Carlton Grove Residence 3400 SW 27 Ave 10...,Ritz Carlton Grove Residence,3400 SW 27 Ave 1002 Miami,"(25.7304471, -80.2380563)",25.730447,-80.238056
10,0Aria on the Bay\r488 NE 18 St 4115\rMiami,2/13/2023,48,"$2,250,000.00",2018,"$1,114.97",Nunziatina Cavallo,G & C Global Realty LLC,Abraham Elbaz,"The Real Estate Lab, Inc.",0Aria on the Bay 488 NE 18 St 4115 Miami,,0Aria on the Bay 488 NE 18 St 4115 Miami,"(25.793483, -80.1869722)",25.793483,-80.186972


In [9]:
df.at[10,'building_name']=('Aria on the Bay')
df.at[10,'address_only']=('488 NE 18 St 4115 Miami')

## Format Data

In [10]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [11]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

In [12]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

## HTML Popup Formatter

In [13]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [14]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Listing Broker']
    buyers_agent = row['Buyer Agent']
    psf = row['Sq. Ft.']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>{}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [15]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February 12th - February 18th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [16]:
m.save('index.html')

## Data snagger

In [17]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [19]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Carillon North Tower , 6899 Collins Ave 3104 Miami Beach | Price $3,995,000 | $1,496 psf | Listing agent: Alex Algarin with Compass Florida LLC | Buyer's agent: Alex Algarin with Compass Florida LLC | Days on market: 76
Least Expensive
Aria on the Bay, 488 NE 18 St 4115 Miami | Price $2,250,000 | $1,115 psf | Listing agent: Nunziatina Cavallo with G & C Global Realty LLC | Buyer's agent: Abraham Elbaz with The Real Estate Lab, Inc. | Days on market: 48
Highest Price Per Square Foot
Waterside Towers Condo , 2450 NE 135 St 907 North Miami | Price $2,749,000 | $3,072 psf | Listing agent: Mario Chirino with Douglas Elliman | Buyer's agent: Elian Garcia Hernandez with Canvas Real Estate | Days on market: 55
Lowest Price Per Square Foot
Space , 01 7934 West Dr 501 North Bay Village | Price $2,400,000 | $645 psf | Listing agent: Vicky Rua with Compass Florida LLC | Buyer's agent: Ramon Dominguez with Lifestyle International Realty | Days on market: 73
Most Days on Market
One Pa

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [20]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [21]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_02182023
